#### IndividualTokenRefinerBlock类

实例为一个残差网络，负责MultiheadSelfAttention计算及条件和时间步信息的注入。数据流如下图所示

In [ ]:
def forward(self, x: torch.Tensor, c: torch.Tensor,  # timestep_aware_representations + context_aware_representations
            attn_mask: torch.Tensor = None,):
        gate_msa, gate_mlp = self.adaLN_modulation(c).chunk(2, dim=1)

        # 第一阶段：使用x计算self_attention得分，并经过msa门控，与x残差连接
        norm_x = self.norm1(x)
        qkv = self.self_attn_qkv(norm_x)
        q, k, v = rearrange(qkv, "B L (K H D) -> K B L H D", K=3, H=self.heads_num)
        # Apply QK-Norm if needed
        q = self.self_attn_q_norm(q).to(v)
        k = self.self_attn_k_norm(k).to(v)
        attn = attention(q, k, v, mode="torch", attn_mask=attn_mask)#这里用mask

        x = x + apply_gate(self.self_attn_proj(attn), gate_msa)  # x + x*gate

        # 第二阶段：第一阶段输出的x通过FFN，再经过mlp门控，与x残差连接
        # FFN Layer
        x = x + apply_gate(self.mlp(self.norm2(x)), gate_mlp)

        return x

![TokenRefiner块示意图](./img/hydit_tokenrefiner.svg)

#### HYVideoDiffusionTransformer类

模型的DiT骨架。初始化时有一些嵌入组件和关键层,其中`hidden_size=3072`

- `img_in = PatchEmbed(patch_size=[1, 2, 2], in_channels=4, hidden_size)`: 将图像潜码用卷积分割成`[1, 2, 2]`大小的块并整形为

$$[B,C,T,H,W]\rightarrow [B,L_{img},D_{hidden}]$$

- `txt_in = SingleTokenRefiner(text_states_dim, hidden_size, heads_num, depth=2)`: 使用时间`t`通过adaLN得到的门控参数调制文本嵌入，并匹配维度

$$[B,L_{txt},D_{txt}]\rightarrow [B,L_{txt},D_{hidden}]$$

- `time_in = TimestepEmbedder(hidden_size, get_activation_layer("silu"))`: 通过MLP，匹配维度

$$[B,]\rightarrow [B,D_{hidden}]$$

- `vector_in = MLPEmbedder(text_states_dim_2, hidden_size)`: 通过MLP，匹配维度

$$[B,D_{txt2}] \rightarrow [B,D_{hidden}]$$

- `modulateDiT`: 一个MLP，输入$vector_{in}+time_{in}$, 学习`shift`,`scale`和`gate`参数
- `double_blocks`: 双数据流DiT块.使用`modulateDiT`输出的参数分别调制`img`和`txt`,`img_mod`和`txt_mod`的`factor`均是6
- `single_block`: 单数据流DiT块，`img`和`txt`作为整体拼接后输入.与double块功能类似，但只在注意力计算前调制.`mod`的`factor`是3,分别为`mod_shift`, `mod_scale`,  `mod_gate`.若使用token_replace,则会多出一组`tr_mod`参数
- `final_layer`: 调制层归一后的`x`, 再经线性投影到$P*P*C_{out}$, 匹配unpatchify.

图像潜码的输入$[B,C,T_{Origin},H_{Origin},W_{Origin}] \stackrel{3D卷积,kernel=stride=[1,2,2]}{\longrightarrow} [B,D_{hidden},T/1,H/2,W/2] \stackrel{flatten+转置}{\longrightarrow} [B, L_{img}, D_{hidden}] \stackrel{Final Layer}{\longrightarrow} [B,p_t × p_h × p_w × C_{out},T/1,H/2,W/2]$ 

$\stackrel{reshape+einsum重排}{\longrightarrow} [B,C,T_{Origin},H_{Origin},W_{Origin}]$

调制过程可写作
$$Y=X⊙(1+\gamma_{scale})+\beta_{shift}$$
`unsqueeze(1)`语句和广播机制会将参数乘到**每一个token上**去。若启用token_replace，第一帧和其余帧则会分别使用来自**时间步0**和**时间步t**的信息调制.注意，`text2`作为引导只是可选的

门控机制可写作
$$Z'=Z⊙(1+tanh(\zeta{gate}))$$
其中$Z$可以来自于`img`和`txt`的**自注意力**得分(是Q,K,V运算后的结果)，或来自于`img`和`txt`的MLP部分。

在double块中会连续先后使用两个不同的门，第一个门控制自注意力得分对自身的**增量改变**，第二个门控制MLP对自身的增量改变。

在single块中，门控作用于attn和mlp并行合并后的结果。某种意义上也是一种"双路"，分两路的不是数据，而是连接。

在forward方法中，首先使用`TimestepEmbedder`和`SingleTokenRefiner`得到时间嵌入`vec`和文本嵌入`vec_2`,并将后者加到`vec`上。若使用CFG，则`guidance`也会通过时间嵌入器加到`vec`中。若启用"token_replace"，则额外再做一次`token_replace_vec`(使用`<tensor>[0,]`生成时间嵌入).

接着，使用`PatchEmbed`嵌入图片为`img`,将`img`和`vec`循环过double块

```
double_block_args = [img, txt, vec,
                cu_seqlens_q, cu_seqlens_kv, max_seqlen_q, max_seqlen_kv,
                freqs_cis,
                self.i2v_condition_type, token_replace_vec, frist_frame_token_num,]
img, txt = block(*double_block_args)
```

接着，将`x = torch.cat((img, txt), 1)`聚合，过single块，并最终提取img

```
x = single_block(*single_block_args)
img = x[:, :img_seq_len, ...]
```

将它经过`final_layer`调制和`unpatchify`，得到最终的`img_latents`用于VAE解码.

#### MMDoubleStreamBlock

图像潜码`x`和条件`txt`分别层归一、调制，再合起来作注意力，得到结果后按累计序列长度拆分为`img`和`txt`，并再次调制后输出。图像和条件各四个调制参数和2个门控，结构如下图所示

来自论文[SD3](https://arxiv.org/abs/2403.03206)

![MMDouble块示意图](./img/hydit_dit2.png)

(暂时不解读，占位...与FlashAttention的优化相关)

In [ ]:
def get_cu_seqlens(text_mask, img_len):
    """Calculate cu_seqlens_q, cu_seqlens_kv using text_mask and img_len

    Args:
        text_mask (torch.Tensor): the mask of text
        img_len (int): the length of image

    Returns:
        torch.Tensor: the calculated cu_seqlens for flash attention
    """
    batch_size = text_mask.shape[0]
    text_len = text_mask.sum(dim=1)
    max_len = text_mask.shape[1] + img_len

    cu_seqlens = torch.zeros([2 * batch_size + 1], dtype=torch.int32, device="cuda")

    for i in range(batch_size):
        s = text_len[i] + img_len
        s1 = i * max_len + s
        s2 = (i + 1) * max_len
        cu_seqlens[2 * i + 1] = s1
        cu_seqlens[2 * i + 2] = s2

    return cu_seqlens

#### MMSingleStreamBlock

图像潜码`img`和条件`txt`先concat，归一调制，再用`Linear1`分成两半`qkv`和`mlp`，前一半做注意力，后一半经过mlp，两者经过`Linear2`得到`output`后经过门控残差结构输出。共2个调制参数和1个门控.

与Double块最大的不同是,Single块的图像和条件的**调制参数由同一个MLP输出**，且经过注意力后不再进行第二次调制。结构如下图所示

来自论文[Scaling Vision Transformers to 22 Billion Parameters](https://arxiv.org/abs/2302.05442)

![MMSingle块示意图](./img/hydit_dit1.png)

#### 回答部分遗留的问题

1. UNet中cross_attention出现的位置、输入输出的数量和形状

    全程并无用于去噪的UNet，它只起到VAE的作用，且只会用到一次，并不是时间开销的大头，其中更没有注意力计算，它只负责编码和解码。注意力计算位于DiT块中，由于图像和文本嵌入被拼接(并分别作用了rope编码)，自注意力得分形如$[B,L_{img+txt},D_{hidden}]$(使用`flash_attn_varlen_func`来分块计算)。它只是一个中间量，它负责**更新**潜变量`img`。

    图像潜码`img`和条件提示`txt`的特征交流***只***位于单流DiT块中。double块(20层)只有自注意力，而single块(40层)只有在最后输出前的**Linear2**完成特征交流，没有交叉注意力机制。`img`和`txt`在$L$维拼接后，形状变化首先是

   $[B,L_{img+txt},N_{heads},D_{perhead}]\stackrel{prenorm+Linear1}{\longrightarrow}[B, L, D_{hidden} * 3 + mlp_{hidden\_dim}(D_{hidden} * 4)]$,

   至此分为$[B,L_{img+txt},D_{hidden}]$的q,k,v和$[B, L_{img+txt}, D_{hidden} * 4]$的mlp四部分.

   将attn和激活后的mlp沿$D_{hidden}$维度重新拼接，并通过Linear2，重新回到$[B,L_{img+txt},D_{hidden}]$